 <h1 style="text-align: center; color: RED">RFM İLE MÜŞTERİ SEGMENTASYONU</h1> 

<span style="color: blue; font-weight: bold;">1-İŞ PROBLEMİ</span> 

Bir e-ticaret şirketi müşterilerini segmentlere ayırarak bu segmentlere göre pazarlama stratejileri belirlemek istiyor.

<span style="color: blue; font-weight: bold;">2-VERİ SETİ HİKAYESİ</span> 

Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.Hediyelik eşya satan bir mağazadır ve müşterileri toptan alışveriş yapmaktadır.

**Değişkenler**

* InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
* StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
* Description: Ürün ismi
* Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
* InvoiceDate: Fatura tarihi ve zamanı.
* UnitPrice: Ürün fiyatı (Sterlin cinsinden)
* CustomerID: Eşsiz müşteri numarası
* Country: Ülke ismi. Müşterinin yaşadığı ülke.


<span style="color: blue; font-weight: bold;">3-VERİYİ ANLAMA</span> 

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option('display.expand_frame_repr',False)
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float.format', lambda x : '%.3f' % x)

In [4]:
df_ = pd.read_excel("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\3.CRM Analytics\\resim\\online_retail_II.xlsx", sheet_name='Year 2009-2010')

In [201]:
df = df_.copy() #her defasında veri setini okumamak için kopyasını aldık

In [8]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [10]:
df.shape

(525461, 8)

In [12]:
df.isnull().sum() #eksik değer kontrolü

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [14]:
#eşsiz ürün sayısı
df['Description'].nunique()

4681

In [16]:
#hangi üründen kaçar tane satılmış 
df['Description'].value_counts().head()

Description
WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: count, dtype: int64

In [18]:
#hangi üründen toplam kaçar tane sipariş verilmiş?
df.groupby('Description').agg({'Quantity':'sum'}) #eksi değerleri suan görmezden geliyoruz.

,Quantity
Description,
21494,-720
22467,-2
22719,2
DOORMAT UNION JACK GUNS AND ROSES,179
3 STRIPEY MICE FELTCRAFT,690
...,...
wet/smashed/unsellable,-700
wonky bottom/broken,-35
wrong code,-258


In [20]:
df.groupby('Description').agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [22]:
#eşsisiz fatura sayısı
df['Invoice'].nunique()

28816

In [113]:
#fatura başına toplam ne kadar kazanılmıştır.
df['TotalPrice'] = df['Quantity'] * df['Price'] #fatura başına kazanç

In [28]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom,30.000


In [30]:
df.groupby('Invoice').agg({'TotalPrice':'sum'}) #faturanın toplam fiyatı

,TotalPrice
Invoice,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240
...,...
C538121,-12.750
C538122,-1.250
C538123,-7.500


<span style="color: blue; font-weight: bold;">4-VERİYİ HAZIRLAMA</span> 

In [32]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

Veri setinde müşteri ıd ve ürün tanımlarında eksik değerler gözlemlenmkete. Customer ıd değişkeni her bir müşteriye ait ıd oldugu için ve bunu herhangi bir değer ile dolduramayacağımız için bu aşamada eksik değerleri veri setinden siliyoruz. Zaten veri seti gözlem bakımından zengin.

In [97]:
df.dropna(inplace=True)#eksik değerleri silmeye yarar

In [99]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [38]:
df.shape

(417534, 9)

Veri setinde invoıce değişkeninde başında C olanlar iadeleri ifade etmektedir. Bu iadeler eksi değerler görmemize sebep oluyor.

In [115]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,407695.000,13.587,1.000,2.000,5.000,12.000,19152.000,96.842
InvoiceDate,407695,2010-07-01 10:10:10.782177792,2009-12-01 07:45:00,2010-03-26 14:01:00,2010-07-09 15:46:00,2010-10-14 17:09:00,2010-12-09 20:01:00,NaN
Price,407695.000,3.294,0.000,1.250,1.950,3.750,10953.500,34.757
Customer ID,407695.000,15368.504,12346.000,13997.000,15321.000,16812.000,18287.000,1679.796
TotalPrice,407695.000,21.663,0.000,4.950,11.900,19.500,15818.400,77.147


Dolayısıyla iade olan faturalı veri setinden çıkarmamız gerekiyor.

In [103]:
df[df['Invoice'].str.contains('C', na=False)]#başında c olan faturaları sectik

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.950,16321.000,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.650,16321.000,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.250,16321.000,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.100,16321.000,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.950,16321.000,Australia
...,...,...,...,...,...,...,...,...
524695,C538123,22956,36 FOIL HEART CAKE CASES,-2,2010-12-09 15:41:00,2.100,12605.000,Germany
524696,C538124,M,Manual,-4,2010-12-09 15:43:00,0.500,15329.000,United Kingdom
524697,C538124,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2010-12-09 15:43:00,2.950,15329.000,United Kingdom
524698,C538124,22423,REGENCY CAKESTAND 3 TIER,-1,2010-12-09 15:43:00,12.750,15329.000,United Kingdom


In [105]:
df = df[~df['Invoice'].str.contains('C', na=False)]#başında c olan olmayan faturaları ~ işareti ile şectik ve iadeleri veri setinden cıkardık

In [117]:
df.shape

(407695, 9)

* Bu kısımda aykırı değer analizi yapmadık. Çünkü rfm skorları oluşturduğumzda o aykırı değerler 5 skoru içerisine girecek. Aynı şekilde aykırı değerleri baskıladığımızda da baskıladıgımız değerler 5 skoru ile ifade edileceği için aykırı değer baskılamaya gerek duymadık.

<span style="color: blue; font-weight: bold;">5-RFM METRİKLERİNİN HESAPLANMASI</span> 

Bu kısımda her bir müşteri özelinde recency, frequency,monetary değerlerini hesaplamayı amaçlıyoruz.

Recency:Müşterinin yeniliğini sıcaklığını ifade eder.
Bunun matematiksel karşılığı= **(analizin yapıldığı tarih-ilgili müşterinin son satın alma tarihi)**

Frequency:Müşetirin yaptığı toplam satın alma

Monetary: Müşterinin yaptığı toplam satın almadan kazanılan para

!!!!  Analiz yaptığımız günü belirlememiz gerekir.
İlgili hesaplmaları yapmamız için analizin yapıldıgı günü tanımlamamız gerekir. Bu veri seti 2009-2011 yılları arasında oluşturulan bir veri setidir.Dolayısıyla analizi o dönemde yapamadığımız için veri seti içerisindeki en son tarih hangisi ise bu tarihin üzerine 2 gün koyarız ve analiz yapılan tarih olarak kabul ederiz.Bu tarih üzerinden recency hesaplarız.

In [119]:
df['InvoiceDate'].max() # bu tarihin üzerine 2 gün ekleyerek analiz tarihini belirleyecz.

Timestamp('2010-12-09 20:01:00')

In [51]:
analysis_date = df['InvoiceDate'].max() + dt.timedelta(days=2) #analiz tarihine 2 gün ekledik
analysis_date

Timestamp('2010-12-11 20:01:00')

Farklı yöntemlerle de analiz tarihini belirleyebiliriz.

In [121]:
analysis_date = dt.datetime(2010, 12, 11)#Bu kod, 2010-12-11 tarihini içeren bir datetime (tarih-zaman) nesnesi oluşturur
#today_date = pd.to_datetime('2010-12-11')#bu bir pandas metodudur.

In [123]:
#CustomerID ler coklama durumunda yani aynı ıd^den birden fazla var groupby ile bunu tekilleştirdik.
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (analysis_date - date.max()).days, #date incvoicedate'teki her bir satırı temsil ediyor
                               'Invoice': lambda num: num.nunique(),
                               'TotalPrice':'sum'})
rfm

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930
...,...,...,...
18283.000,18,6,641.770
18284.000,67,1,461.680
18285.000,296,1,427.000


In [64]:
rfm.columns 

Index(['InvoiceDate', 'Invoice', 'TotalPrice'], dtype='object')

In [125]:
rfm.columns = ['Recency','Frequency','Monetary']

#rfm = rfm.rename(columns={"customer_id": "Customer_ID", "recency": "Recency"}) değişkne isimlerini bu yöntemle de değitirebilirsin

In [127]:
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [129]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
Frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
Monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.803,349164.350


* Monetary değerinde 0 olması istenmeyen bir durumdur. 0' dan büyük olan değerleri seçmeliyiz

In [131]:
rfm = rfm[rfm['Monetary'] > 0]

In [133]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
Frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
Monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


In [135]:
rfm.shape

(4312, 3)

<span style="color: blue; font-weight: bold;">6-RFM SKORLARININ HESAPLANMASI</span>

!!!Recency küçük ise skoru büyük olacak. Frequency ve monetary büyük ise skoruda büyük olacak.

qcut çeyreklklere göre bölme işlemi yapar. Bir değişkeni küçükten büyüğe sıralar ve verdiğimiz sayıya göre  parçalara böler ve bizim girdiğimiz aralıklara göre bunları sınıflandırır.

In [137]:
rfm['recency_score'] = pd.qcut(rfm['Recency'], 5, labels = [5, 4, 3, 2, 1])

C:\Users\ASLI\AppData\Local\Temp\ipykernel_6172\3214870952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['recency_score'] = pd.qcut(rfm['Recency'], 5, labels = [5, 4, 3, 2, 1])


In [139]:
rfm.head()

,Recency,Frequency,Monetary,recency_score
Customer ID,,,,
12346.000,165,11,372.860,2
12347.000,3,2,1323.320,5
12348.000,74,1,222.160,2
12349.000,43,3,2671.140,3
12351.000,11,1,300.930,5


In [141]:
rfm['monetary_score'] = pd.qcut(rfm['Monetary'], 5, labels = [1, 2, 3, 4, 5])

C:\Users\ASLI\AppData\Local\Temp\ipykernel_6172\1963276945.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['monetary_score'] = pd.qcut(rfm['Monetary'], 5, labels = [1, 2, 3, 4, 5])


In [143]:
rfm.head()

,Recency,Frequency,Monetary,recency_score,monetary_score
Customer ID,,,,,
12346.000,165,11,372.860,2,2
12347.000,3,2,1323.320,5,4
12348.000,74,1,222.160,2,1
12349.000,43,3,2671.140,3,5
12351.000,11,1,300.930,5,2


In [145]:
rfm['frequency_score'] = pd.qcut(rfm['Frequency'], 5, labels = [1, 2, 3, 4, 5])

ValueError: Bin edges must be unique: Index([1.0, 1.0, 2.0, 3.0, 6.0, 205.0], dtype='float64', name='Frequency').
You can drop duplicate edges by setting the 'duplicates' kwarg

* Hata verme sebebi paraçalara bölmek istediğinde örneğin 1 değerinin birden fazla sınıfta olması. Örneğin 1 değeri hem skor1 de hem 2 de var gibi düşünebiliriz. Çok fazla tekrar eden değerler oldugu için böldüğünde ceyrek değerlere düşün değerler aynı olmuş.

In [147]:
rfm['frequency_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels = [1, 2, 3, 4, 5])

C:\Users\ASLI\AppData\Local\Temp\ipykernel_6172\751090854.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['frequency_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels = [1, 2, 3, 4, 5])


*  qcut, veriyi eşit sayıda gruba bölmeye çalışır, ama aynı değerde çok sayıda gözlem varsa bu teknik olarak mümkün olmayabilir.  rank(method='first') bu metod aynı değerdeki gözlemleri bile farklı sıralar vererek ayırır.Bu sayede qcut fonksiyonu için tüm değerler benzersiz hale gelir.

In [150]:
rfm.head()


,Recency,Frequency,Monetary,recency_score,monetary_score,frequency_score
Customer ID,,,,,,
12346.000,165,11,372.860,2,2,5
12347.000,3,2,1323.320,5,4,2
12348.000,74,1,222.160,2,1,1
12349.000,43,3,2671.140,3,5,3
12351.000,11,1,300.930,5,2,1


In [152]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4312 entries, 12346.0 to 18287.0
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Recency          4312 non-null   int64   
 1   Frequency        4312 non-null   int64   
 2   Monetary         4312 non-null   float64 
 3   recency_score    4312 non-null   category
 4   monetary_score   4312 non-null   category
 5   frequency_score  4312 non-null   category
dtypes: category(3), float64(1), int64(2)
memory usage: 148.0 KB


In [154]:
rfm['RF_SCORE'] = rfm['recency_score'] + rfm['frequency_score'] #iki categorik değişkeni toplyamazsın hatası aldık

TypeError: unsupported operand type(s) for +: 'Categorical' and 'Categorical'

In [156]:
rfm['RF_SCORE'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)

C:\Users\ASLI\AppData\Local\Temp\ipykernel_6172\593758803.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['RF_SCORE'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)


In [158]:
rfm.head()

,Recency,Frequency,Monetary,recency_score,monetary_score,frequency_score,RF_SCORE
Customer ID,,,,,,,
12346.000,165,11,372.860,2,2,5,25
12347.000,3,2,1323.320,5,4,2,52
12348.000,74,1,222.160,2,1,1,21
12349.000,43,3,2671.140,3,5,3,33
12351.000,11,1,300.930,5,2,1,51


In [160]:
rfm[rfm['RF_SCORE']=='55'] #ŞAMPİYON MÜŞTERİLER

,Recency,Frequency,Monetary,recency_score,monetary_score,frequency_score,RF_SCORE
Customer ID,,,,,,,
12415.000,11,7,19543.840,5,5,5,55
12431.000,9,13,4370.520,5,5,5,55
12471.000,10,49,20139.740,5,5,5,55
12472.000,5,13,11308.480,5,5,5,55
12474.000,14,13,5048.660,5,5,5,55
...,...,...,...,...,...,...,...
18225.000,1,15,7545.140,5,5,5,55
18226.000,14,15,6650.830,5,5,5,55
18229.000,2,10,3526.810,5,5,5,55


<span style="color: blue; font-weight: bold;">7-RFM SEGMENTLERİNİN OLUŞTURULMASI</span>


Regex (Regular Expression), metin arama, eşleştirme, değiştirme ve doğrulama işlemleri için kullanılan güçlü bir araçtır.
Genellikle veri işleme ve manipülasyonu gerektiren durumlarda kullanılır.
Regex, belirli desenlere uyan metinleri aramak, değiştirmek, veya çıkarmak için oldukça faydalıdır.
Sözlüğünüzdeki her bir regex deseni, müşteri davranışlarını belirten bir skoru ifade eder. Segmentlerin anlamını şu şekilde açıklayabiliriz:

r'[1-2][1-2]':Bu desen, 1 ile 2 arasında iki haneli sayıları ifade eder, yani 11, 12, 21, 22 gibi.

Segment adı: hibernating (uykuda olan müşteriler).

Bu, müşterilerin çok uzun süredir işlem yapmadığını ve şu anda aktif olmadığını gösterebilir.


In [166]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [169]:
rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex = True)

C:\Users\ASLI\AppData\Local\Temp\ipykernel_6172\1557666392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex = True)


Bu kod, rfm adlı DataFrame'deki RF_SCORE sütunundaki her bir değeri, seg_map sözlüğüne dayanarak belirli segmentlere dönüştürmek için kullanılır
replace() metodu, belirtilen seg_map sözlüğüne göre, RF_SCORE sütunundaki değerleri belirli segmentlerle değiştirecektir.
regex=True parametresi, anahtarların regex (düzenli ifade) desenleri olduğunu belirtir. Bu, belirli desenlere uyan değerlerin eşleştirilerek yerine yeni segment isminin yazılmasını sağlar.
Örneğin, rfm['RF_SCORE'] içinde "15" varsa, bu "cant_loose" segmentine dönüşecektir.

In [171]:
rfm.head()

,Recency,Frequency,Monetary,recency_score,monetary_score,frequency_score,RF_SCORE,segment
Customer ID,,,,,,,,
12346.000,165,11,372.860,2,2,5,25,cant_loose
12347.000,3,2,1323.320,5,4,2,52,potential_loyalists
12348.000,74,1,222.160,2,1,1,21,hibernating
12349.000,43,3,2671.140,3,5,3,33,need_attention
12351.000,11,1,300.930,5,2,1,51,new_customers


In [175]:
#Ssegmentleri analiz etmek içşin bunu yaptık
rfm[['segment','Recency','Frequency','Monetary']].groupby('segment').agg(['count','mean'])

Recency         Frequency        Monetary         
                      count    mean     count   mean    count     mean
segment                                                               
about_to_sleep          343  53.819       343  1.201      343  441.320
at_Risk                 611 152.159       611  3.074      611 1188.878
cant_loose               77 124.117        77  9.117       77 4099.450
champions               663   7.119       663 12.554      663 6852.264
hibernating            1015 213.886      1015  1.126     1015  403.978
loyal_customers         742  36.287       742  6.830      742 2746.067
need_attention          207  53.266       207  2.449      207 1060.357
new_customers            50   8.580        50  1.000       50  386.199
potential_loyalists     517  18.793       517  2.017      517  729.511
promising                87  25.747        87  1.000       87  367.087

In [177]:
rfm[rfm['segment']=='need_attention']

,Recency,Frequency,Monetary,recency_score,monetary_score,frequency_score,RF_SCORE,segment
Customer ID,,,,,,,,
12349.000,43,3,2671.140,3,5,3,33,need_attention
12369.000,49,3,1791.150,3,4,3,33,need_attention
12371.000,45,3,2179.420,3,5,3,33,need_attention
12374.000,57,3,2246.290,3,5,3,33,need_attention
12389.000,38,3,1433.330,3,4,3,33,need_attention
...,...,...,...,...,...,...,...,...
18078.000,61,2,508.470,3,3,3,33,need_attention
18112.000,54,2,423.270,3,2,3,33,need_attention
18136.000,45,2,847.030,3,3,3,33,need_attention


In [179]:
rfm[rfm['segment']=='cant_loose']

,Recency,Frequency,Monetary,recency_score,monetary_score,frequency_score,RF_SCORE,segment
Customer ID,,,,,,,,
12346.000,165,11,372.860,2,2,5,25,cant_loose
12380.000,101,7,6951.490,2,5,5,25,cant_loose
12482.000,212,29,23691.400,1,5,5,15,cant_loose
12510.000,95,7,4195.450,2,5,5,25,cant_loose
12891.000,94,8,509.500,2,3,5,25,cant_loose
...,...,...,...,...,...,...,...,...
18051.000,270,7,2282.280,1,5,5,15,cant_loose
18064.000,82,9,3681.030,2,5,5,25,cant_loose
18094.000,82,6,3374.220,2,5,5,25,cant_loose


In [181]:
rfm[rfm['segment']=='cant_loose'].index #sadece müşteri ıd lerine erişmek istersek

Index([12346.0, 12380.0, 12482.0, 12510.0, 12891.0, 12932.0, 13044.0, 13313.0,
       13680.0, 13782.0, 13799.0, 13856.0, 14025.0, 14063.0, 14160.0, 14221.0,
       14548.0, 14607.0, 14685.0, 14745.0, 15003.0, 15013.0, 15015.0, 15125.0,
       15141.0, 15222.0, 15306.0, 15321.0, 15359.0, 15369.0, 15372.0, 15443.0,
       15538.0, 15607.0, 15633.0, 15722.0, 15751.0, 15754.0, 15768.0, 15911.0,
       15912.0, 16027.0, 16032.0, 16158.0, 16177.0, 16197.0, 16335.0, 16467.0,
       16631.0, 16742.0, 16743.0, 16875.0, 16986.0, 17021.0, 17032.0, 17092.0,
       17113.0, 17157.0, 17188.0, 17230.0, 17268.0, 17426.0, 17448.0, 17454.0,
       17512.0, 17578.0, 17602.0, 17651.0, 17940.0, 17969.0, 17988.0, 18009.0,
       18051.0, 18064.0, 18094.0, 18251.0, 18258.0],
      dtype='float64', name='Customer ID')

In [191]:
new_df = pd.DataFrame()

In [193]:
new_df['new_customer_id']= rfm[rfm['segment']=='new_customers'].index
new_df

,new_customer_id
0,12351.000
1,12385.000
2,12386.000
3,12427.000
4,12441.000
5,12538.000
6,12686.000
7,12738.000
8,12763.000
9,12767.000


In [195]:
new_df['new_customer_id'] = new_df['new_customer_id'].astype(int)

In [197]:
new_df

,new_customer_id
0,12351
1,12385
2,12386
3,12427
4,12441
5,12538
6,12686
7,12738
8,12763
9,12767


In [ ]:
new_df.to_csv('new_customer.csv')#yukarıdaki lisyei dışarı aktardık
rfm.to_csv('rfm.csv')#tüm dosyayıda aktarabilriiz

 <h3 style="text-align: center; color: RED">SÜRECİN FONKSİYONLAŞTIRILMASI</h3> 

In [217]:
def create_rfm(dataframe, csv=False):
    dataframe['TotalPrice'] = dataframe['Quantity'] * dataframe['Price']
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe['Invoice'].str.contains('C', na=False)]

    #RFM METRİKLERİNN HESAPLANMASI
    today_date = dt.datetime(2011,12,11)
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date-date.max()).days,
                                               'Invoice': lambda num: num.nunique(),
                                               'TotalPrice':'sum'})
    rfm.columns = ['Recency','Frequency','Monetary']
    rfm = rfm[rfm['Monetary'] > 0]

    #RFM SKORLARININ HESAPLANMASI
    rfm['recency_score'] = pd.qcut(rfm['Recency'], 5, labels = [5,4,3,2,1])
    rfm['frequency_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels = [1,2,3,4,5])
    rfm['monetary_score'] = pd.qcut(rfm['Monetary'], 5, labels = [1,2,3,4,5])

    rfm['RF_SCORE'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)

    #SEGMENTLERİN İSİMLENDİRİLMESİ
    seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
    }

    rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)
    rfm = rfm[['Recency','Frequency','Monetary','RF_SCORE','segment']]
    rfm.index = rfm.index.astype(int)

    if csv:
        rfm.to_csv('rfm.csv')
    return rfm

create_rfm(df)

,Recency,Frequency,Monetary,RF_SCORE,segment
Customer ID,,,,,
12346,530,11,372.860,25,cant_loose
12347,368,2,1323.320,52,potential_loyalists
12348,439,1,222.160,21,hibernating
12349,408,3,2671.140,33,need_attention
12351,376,1,300.930,51,new_customers
...,...,...,...,...,...
18283,383,6,641.770,45,loyal_customers
18284,432,1,461.680,32,about_to_sleep
18285,661,1,427.000,12,hibernating


In [231]:
df.isnull().values

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])